In [15]:
import konlpy
from konlpy.tag import Kkma, Komoran, Hannanum, Okt
from konlpy.utils import pprint

In [16]:
okt = Okt()
kkma = Kkma()

JVMNotFoundException: No JVM shared library file (jvm.dll) found. Try setting up the JAVA_HOME environment variable properly.

In [ ]:



import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

import re
from collections import Counter

import warnings
warnings.simplefilter("ignore")

from wordcloud import WordCloud
import matplotlib.pyplot as plt


from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

# 기초 불용어 처리
from nltk.corpus import stopwords

# 경고 뜨지 않게 설정
import warnings
warnings.filterwarnings('ignore')

# 형태소 벡터를 생성하기 위한 라이브러리
from sklearn.feature_extraction.text import CountVectorizer
# 형태소 벡터를 학습 벡터로 변환한다.
from sklearn.feature_extraction.text import TfidfTransformer

# 워드 클라우드를 위한 라이브러리
from collections import Counter
# import pytagcloud
from IPython.display import Image

# 저장
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Filtered path 불러오기 
path = r"/content/drive/MyDrive/2023 T아카데미/개인프로젝트/Filtered.xlsx"
df = pd.read_excel(path )

In [ ]:
df.head()


In [ ]:
count_text = len(df[df["Text"] != 0])
print(count_text)

In [ ]:
# 불용어 path 불러오기 

path = r"/content/drive/MyDrive/2023 T아카데미/개인프로젝트/kor_stopwords.xlsx"
df_stopwords = pd.read_excel(path)

In [ ]:
df_stopwords

In [ ]:
stopwords_list = df_stopwords[0].tolist()

In [ ]:
print(stopwords_list)

In [ ]:
sample_text = df.iloc[0,9] 
print(sample_text)
print(type(sample_text))

In [ ]:
# 정규화
def apply_regular_expression(Text):
    kor = re.compile('[^ ㄱ-ㅣ 가-힣]')  # 한글 추출 규칙: 띄어 쓰기(1 개)를 포함한 한글
    result = kor.sub("",Text) # 위에 설정한 kor 규칙을 text에 적용시킴 
    return result

In [ ]:
def apply_stopwords(sample_text):
  #  1) 공백 제거
    remove_blank = re.compile(r'\W*\b\w\b')
    remove_mark = remove_blank.sub('', sample_text).replace('.', '')
    # print(remove_mark)
    
    # 2) 정규화
    regular = apply_regular_expression(remove_mark)
    # print(regular)
    
    # 3) 불용어 처리 및 조사 제거
    tok = okt.morphs(regular)
    tok_str = ' '.join(tok)
    # print(tok_str)

    shortword = re.compile(r'\W*\b\w{1}\b')
    third_processed = shortword.sub('', tok_str)
    third_processed = okt.morphs(third_processed)
    # print(third_processed)

    result = []
    for w in third_processed:
        if w not in stopwords_list:
            result.append(w)
    return result

In [ ]:
apply_stopwords(df.iloc[0,9])

In [ ]:
apply_stopwords(df.iloc[0,9])

In [ ]:
# 초기화
df['FilteredText'] = df['Text']

In [ ]:
df

In [ ]:
df

In [ ]:
def filteredtext(df):  
  for i in range(len(df)-1):
    if df.iloc[i,9] != 0:
      print(i)
      df.iloc[i,9] = apply_stopwords(df.iloc[i,9])      
      
         

In [ ]:
filteredtext(df)
# ValueError: Must have equal len keys and value when setting with an iterable 

In [ ]:
# 7167 rows × 10 columns

In [ ]:
def filteredtext(df):  
  for i in range(len(df)):
    if df.iloc[i,9] != 0:
      df.at[i, 'FilteredText'] = apply_stopwords(df.iloc[i,9])
  

In [ ]:
filteredtext(df)
# 

In [ ]:
df

In [ ]:
df.tail()

In [ ]:
df.to_excel("Filtered_stopwords.xlsx")

In [ ]:
# FilteredText
kkma = konlpy.tag.Kkma() #형태소 분석기 꼬꼬마(Kkma)
nouns = df['FilteredText'].apply(lambda x: kkma.nouns(str(x)))
nouns

In [ ]:
nouns = nouns.explode()
nouns

In [ ]:
df_word = pd.DataFrame({'word' : nouns})
df_word['count'] = df_word['word'].str.len()
df_word.word=df_word.word.astype(str)
df_word = df_word.query('count > = 2')
df_word 

In [ ]:
size_exist=df_word.index[df_word["word"]=="사이즈"].tolist()

In [ ]:
print(size_exist)

In [ ]:
for i in size_exist:
    print("사이즈가 언급되는 리뷰")
    print(df.iloc[int(i),5])
    print(df.iloc[i,9])

In [ ]:
price_exist=df_word.index[df_word["word"]=="가격"].tolist()

In [ ]:
for i in price_exist:
    print("가격이 언급되는 리뷰")
    print(df.iloc[int(i),5])
    print(df.iloc[i,9])

In [ ]:
shipment_exist=df_word.index[df_word["word"]=="배송"].tolist()

In [ ]:
for i in shipment_exist:
    print("배송이 언급되는 리뷰")
    print(df.iloc[int(i),5])
    print(df.iloc[i,9])

In [ ]:
df_word = df_word.groupby('word', as_index = False).count().sort_values('count', ascending = False)
df_word

In [ ]:
dic_word = df_word.set_index('word').to_dict()['count']
dic_word

In [ ]:
font_path = r"/content/drive/MyDrive/2023 T아카데미/개인프로젝트/작업본/NanumGothic.ttf"

wc = WordCloud(random_state = 123, width = 400,font_path = font_path,
               height = 400, background_color = 'white')

img_wordcloud = wc.generate_from_frequencies(dic_word)

plt.figure(figsize = (10, 10)) # 크기 지정하기
plt.axis('off') # 축 없애기
plt.imshow(img_wordcloud) # 결과 보여주기
plt.savefig('filtered_review_text_wordcloud') # 파일 저장

# additional stopwords : 진짜, 완전, 감사, 겠어, 하세

In [ ]:
font_path = r"/content/drive/MyDrive/2023 T아카데미/개인프로젝트/작업본/NanumGothic.ttf"

img = Image.open(r"C:\Users\NT550009\Desktop\11st\t-shirt icon wordcloud.png") #이미지 오픈 
imgArray = np.array(img) # 이미지의 각 셀을 수치로 변환

wc = WordCloud(width = 400,height = 400, font_path = font_path,
               background_color = 'white', mask = imgArray).generate_from_frequencies(dic_word)

plt.figure(figsize = (15, 15)) # 크기 지정하기
plt.imshow(wc ,interpolation = 'bilinear')
plt.axis('off') # 축 없애기
plt.imshow(wc ) # 결과 보여주기

# plt.savefig('filtered_review_text_wordcloud')